In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
from numpy import expand_dims


# Load các model
lstm_model = tf.keras.models.load_model('./Result-ML/final/lstm-5000')
sgan_model = tf.keras.models.load_model('./Result-ML/final/sgan/5000/disc_sup_0070.h5')
meta_learner = joblib.load('./Result-ML/final/meta-learner-rf-5000.joblib')

# Đọc dữ liệu test
full_df = pd.read_csv('../balanced_transaction-from-to-prefix-22-KLTN-newtime.csv')

full_df = full_df.replace(np.nan, -1)

# Chia dataset thành train (80%) và test (20%)
train_df, test_df = train_test_split(full_df, test_size=0.2, random_state=42)

# Tiền xử lý dữ liệu
X_test = test_df.drop(columns=['hash', 'from_address', 'to_address', 'label'], axis=1).values
y_test = test_df['label'].values

# Xử lý cho SGAN
num_padding_sgan = 49 - X_test.shape[1] % 49
X_padded_sgan = np.pad(X_test, ((0, 0), (0, num_padding_sgan)), mode='constant', constant_values=-2)
scaler_sgan = MinMaxScaler()
X_normalized_sgan = scaler_sgan.fit_transform(X_padded_sgan)
X_reshaped_sgan = X_normalized_sgan.reshape(-1, 7, 7)
X_reshaped_sgan = expand_dims(X_reshaped_sgan, axis=-1)

# Xử lý cho LSTM
num_padding_lstm = 900 - X_test.shape[1] % 900
X_padded_lstm = np.pad(X_test, ((0, 0), (0, num_padding_lstm)), mode='constant', constant_values=-2)
scaler_lstm = MinMaxScaler()
X_normalized_lstm = scaler_lstm.fit_transform(X_padded_lstm)
X_reshaped_lstm = X_normalized_lstm.reshape(-1, 30, 30)
X_reshaped_lstm = expand_dims(X_reshaped_lstm, axis=-1)

# Dự đoán
y_pred_sgan = sgan_model.predict(X_reshaped_sgan)
y_pred_sgan = np.argmax(y_pred_sgan, axis=1)
# Đánh giá kết quả SGAN
accuracy = accuracy_score(y_test, y_pred_sgan)
precision = precision_score(y_test, y_pred_sgan)
recall = recall_score(y_test, y_pred_sgan)
f1 = f1_score(y_test, y_pred_sgan)
conf_matrix = confusion_matrix(y_test, y_pred_sgan)
print("####### SGAN #######")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
tn, fp, fn, tp = conf_matrix.ravel()
fpr = fp / (fp + tn)
print(f"False Positive Rate: {fpr:.4f}")

y_pred_lstm = lstm_model.predict(X_reshaped_lstm)
y_pred_lstm = np.argmax(y_pred_lstm, axis=1)
# Đánh giá kết quả LSTM
accuracy = accuracy_score(y_test, y_pred_lstm)
precision = precision_score(y_test, y_pred_lstm)
recall = recall_score(y_test, y_pred_lstm)
f1 = f1_score(y_test, y_pred_lstm)
conf_matrix = confusion_matrix(y_test, y_pred_lstm)
print("####### LSTM #######")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
tn, fp, fn, tp = conf_matrix.ravel()
fpr = fp / (fp + tn)
print(f"False Positive Rate: {fpr:.4f}")

# Chuẩn bị dữ liệu cho meta-learner
scaler_meta = StandardScaler()
X_scaled_meta = scaler_meta.fit_transform(X_test)
meta_features = np.column_stack((X_scaled_meta, y_pred_lstm, y_pred_sgan))

# Dự đoán bằng meta-learner
y_pred_meta = meta_learner.predict(meta_features)

# Đánh giá kết quả
accuracy = accuracy_score(y_test, y_pred_meta)
precision = precision_score(y_test, y_pred_meta)
recall = recall_score(y_test, y_pred_meta)
f1 = f1_score(y_test, y_pred_meta)
conf_matrix = confusion_matrix(y_test, y_pred_meta)

print("####### Meta #######")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

# Tính false positive rate
tn, fp, fn, tp = conf_matrix.ravel()
fpr = fp / (fp + tn)
print(f"False Positive Rate: {fpr:.4f}")

297/297 [==============================] - 0s 940us/step
####### SGAN #######
Accuracy: 0.9231
Precision: 0.9752
Recall: 0.8546
F1-score: 0.9109
Confusion Matrix:
[[5030   95]
 [ 635 3733]]
False Positive Rate: 0.0185
297/297 [==============================] - 6s 18ms/step
####### LSTM #######
Accuracy: 0.9493
Precision: 0.9523
Recall: 0.9368
F1-score: 0.9445
Confusion Matrix:
[[4920  205]
 [ 276 4092]]
False Positive Rate: 0.0400
####### Meta #######
Accuracy: 0.9388
Precision: 0.9907
Recall: 0.8752
F1-score: 0.9294
Confusion Matrix:
[[5089   36]
 [ 545 3823]]
False Positive Rate: 0.0070
